In this notebook, I show the performance of a trained neural network when run within the SAM model. I show three main cases:

1. The training dataset which is the NG-Aqua data post-processed by the SAM model. The post-processing does several things. First, it projects the velocity fields onto the divergence-free subspace. Second, it evolves these base-states 10 minutes into the future, and uses the difference between this new state and the divergence-free initial condition to compute the tendency due to SAM.
2. A passive run with the dynamical core of SAM (w/ no SGS) evolving forward in time, with the neural network producing predictions, but not updating the state.
3. An active run which full couples SAM and the neural network

In [ ]:
import os
import glob
from toolz import get_in, valmap
from uwnet.analysis.sam_debug import *


# plotting libraries
import holoviews as hv
hv.extension('bokeh')

# colorblind friendly defaults
import gnl
gnl.colorblind()

%opts Image[colorbar=True, width=300, height=150](cmap='viridis')

Load debugging data and training data into a single Dataset.

In [ ]:
training_data_path = "../../data/training_data.nc"
ids = {
    'passive': "tender_curie",
    'active':  "deadly_becquerel"
}
paths = valmap(lambda x: os.path.join("../../data", "samNN", x), ids)

def _open_debug(t):
    ds = open_debug_and_training_data(t, paths, training_data_path)
    return t * 20*  30/3600, ds

ds = concat_datasets([_open_debug(t) for t in [0, 5, 10]], name='t')

In [ ]:
ds.Prec.plot(col="tag", row="t", aspect=2, vmax=100)

It looks like the passive estimate of precip is pretty good, but the active scheme seems to show some blow up.

In [ ]:
w_vert = (ds.W * ds.layer_mass).sum('z')
w_vert.plot(col="tag", row="t", aspect=2)

the vertically integrated vertical velocity is a measure of mean upward motion. This plot shows that the active sheme is producing extremely large vertical velocities on very fast time.

In [ ]:
ds.Prec.mean(['x', 'y'])

The precipitation predicted by the neural network in passive mode is also too strong.

In [ ]:
ds.RADSFC.plot(col="tag", row="t", aspect=2)

there are some systematic biases with the predicted radiation. I think this is probably because SAM is not being fed the correct SOLIN.

In [ ]:
ds.SOLIN.isel(t=0, x=0).plot(hue='tag')

Indeed, the SOLIN is not correct. This should be an easy fix in the model, and probably does not explain the fundamental problem with stability that we are seeing though. 

# Composites for high precipitation points

To get some insight, let's composite the vertical velocity, FQT and FSL for precipitation rates within certain bins. These are the bin edges

In [ ]:
bins = [0, 25, 50, 100, 150, 200]

Here is the command to composite these things.

In [ ]:
composite = (ds.groupby_bins('Prec', bins)
              .apply(lambda x: x.groupby('tag')
                     .apply(lambda x: x.groupby('z').mean())
                    ))


In [ ]:
composite.W.plot(y='z', col='tag', hue='Prec_bins')
plt.xlim([-.2, .2])

The structure of vertical velocity is quite different for the points with precip > 150 mm/day for the model with an active NN. Specifically, the vertical velocity has a secondary peak above z=10 km. Note, no point as P > 150 for the passive or training data, so there are no corresponding blue curves.

This change in vertical velocity has a very large influence on FSLI in the stratosphere. In fact, this field has a very unrealistic structure in the active model for lower values of precip.

In [ ]:
(composite.FSLI*86400).plot(y='z', col='tag', hue='Prec_bins')
plt.xlim([-100, 50])

In [ ]:
(composite.FQT*86400).plot(y='z', col='tag', hue='Prec_bins')
plt.xlim([-1, 40])

This change in vertical velocities also effects FQT, but in a less visibly drastic way.

# Conclusions

1. I need to fix the SOLIN computation routine. There is probably something wrong with how I am computing the zenith angle.
2. There is probably some numerical instability involving W-FSL-precipitation in the coupled model.